In [1]:
from swat import *
import swat as sw
from pprint import pprint
%matplotlib inline
import matplotlib
import sys

sys.path.append('C:\\Users\\weshiz\\Documents\\GitHub\\modify\\python-dlpy')
sys.path.append('C:\\Users\\weshiz\\Documents\\GitHub\\python-fcmp')
from dlpy.layers import * 
from dlpy.applications import *
from dlpy import Model, Sequential
from dlpy.utils import *
from dlpy.splitting import two_way_split
from dlpy.images import *
from dlpy.model import *
from src.parser import *
from src.decorator import *
from dlpy.lr_scheduler import *

import numpy as np
import pandas as pd

In [2]:
s = sw.CAS('dlgrd009', 13300)
s.loadactionset('deeplearn')

NOTE: Added action set 'deeplearn'.


[actionset]

 'deeplearn'

+ Elapsed: 0.00352s, user: 0.00285s, sys: 0.000648s, mem: 0.205mb

In [3]:
s.sessionProp.setSessOpt(caslib='CASUSER', cmplib="CASUSER.fcmpfunction")

NOTE: 'CASUSER(weshiz)' is now the active caslib.


+ Elapsed: 0.000168s, user: 8.7e-05s, sys: 7.2e-05s, mem: 0.256mb

### FCMP Function

In [4]:
@out_args('y_out')
@cast_array('srcY', 'weights', 'y_out')
def forward_prop(srcHeight, srcWidth, srcDepth, srcY, weights, y_out):
    y_out[0] = srcY[0]**2 * weights[0] + weights[1]
    return y_out[0]

@out_args('gradient_out', 'srcDeltas_out')
@cast_array('srcY', 'Y', 'weights', 'deltas', 'gradient_out', 'srcDeltas_out')
def back_prop(srcHeight, srcWidth, srcDepth, srcY, Y, weights,
              deltas, gradient_out, srcDeltas_out):
    gradient_out[0] = deltas[0] * (srcY[0] ** 2)
    gradient_out[1] = deltas[0]
    srcDeltas_out[0] = deltas[0] * (2 * weights[0] * srcY[0] + weights[1])
    return

Arguments, srcY, weights, y_out, are casted to array type
Arguments, y_out, are declared as outargs.
Arguments, srcY, Y, weights, deltas, gradient_out, srcDeltas_out, are casted to array type
Arguments, gradient_out, srcDeltas_out, are declared as outargs.


In [5]:
# get FCMP code of forward_prop
forward_fcmp_code = python_to_fcmp(forward_prop, True)

('function forward_prop(srcHeight, srcWidth, srcDepth, srcY[*], weights[*], '
 'y_out[*]);outargs y_out;\n'
 '    y_out[1] = (srcY[1] ** 2 * weights[1] + weights[2]);\n'
 '    return (y_out[1]);\n'
 'endsub;\n')


In [6]:
# get FCMP code of back_prop
backward_fcmp_code = python_to_fcmp(back_prop, True)

('function back_prop(srcHeight, srcWidth, srcDepth, srcY[*], Y[*], weights[*], '
 'deltas[*], gradient_out[*], srcDeltas_out[*]);outargs gradient_out, '
 'srcDeltas_out;\n'
 '    gradient_out[1] = deltas[1] * srcY[1] ** 2;\n'
 '    gradient_out[2] = deltas[1];\n'
 '    srcDeltas_out[1] = deltas[1] * (2 * weights[1] * srcY[1] + weights[2]);\n'
 '    return ;\n'
 'endsub;\n')


In [7]:
# register forward and backward function together
register_fcmp_routines(s, routine_code=forward_fcmp_code + backward_fcmp_code, function_tbl_name='fcmpfunction')

NOTE: Cloud Analytic Services saved the file FCMPFUNCTION.sashdat in caslib CASUSER(weshiz).


### Define a model

In [8]:
s.buildmodel(type='cnn', modeltable=dict(name='fcmp_model', replace=1))
s.addlayer(layer=dict(type="input"), name="input",
           model="fcmp_model")
s.addlayer(layer=dict(type="FCMP", forwardFunc="forward_prop", backwardFunc="back_prop", 
           height=1, width=1, depth=1, nWeights=2), name="fcmp_layer", srcLayers="input",
           model="fcmp_model")
s.addlayer(layer=dict(type="output", error="NORMAL",  includeBias=False, fullConnect=False, act="IDENTITY"), 
           srcLayers='fcmp_layer',
           name="output", model="fcmp_model")

,casLib,Name,Rows,Columns,casTable
0,CASUSER(weshiz),fcmp_model,34,5,"CASTable('fcmp_model', caslib='CASUSER(weshiz)')"


### Create a dataset

In [9]:
# a, b are weights to be learned
a = 3.5
b = 5.1

In [10]:
x = np.random.random(50)
x

array([0.33308802, 0.70717241, 0.67213261, 0.48323607, 0.37469729,
       0.33251719, 0.67096248, 0.24254784, 0.25777387, 0.07585573,
       0.55091958, 0.07513484, 0.62268136, 0.18939629, 0.18061411,
       0.56732067, 0.34303274, 0.35467101, 0.54940535, 0.0098571 ,
       0.45668146, 0.48609047, 0.27879276, 0.10019084, 0.13812319,
       0.16815841, 0.92279466, 0.35668288, 0.42447938, 0.73507596,
       0.80456158, 0.60533332, 0.794088  , 0.50830447, 0.07616976,
       0.34357565, 0.35655163, 0.52023852, 0.11077409, 0.65796409,
       0.12505299, 0.53832619, 0.88139624, 0.92504791, 0.71166064,
       0.42843404, 0.88621664, 0.96625217, 0.91323878, 0.07242452])

In [11]:
noisy = np.random.normal(size=50) / 100
noisy

array([ 0.00792801,  0.00196627, -0.00470699,  0.00166786, -0.00064238,
       -0.01800476, -0.00860187, -0.00585783, -0.01175271,  0.00043938,
       -0.00428229,  0.0060955 , -0.00358657,  0.01114719,  0.00594957,
       -0.0002198 ,  0.00198887, -0.00150801, -0.00037054,  0.002976  ,
        0.00654023,  0.01342725,  0.0159679 , -0.00169169,  0.00244786,
       -0.00046402, -0.01878729,  0.01444742,  0.01162142, -0.01821125,
        0.00631519,  0.00772759, -0.00596985,  0.01044149, -0.00849968,
        0.02220993, -0.00590816,  0.02027319, -0.02643683, -0.00382042,
        0.014522  ,  0.00441466, -0.01096772, -0.00463771, -0.00308117,
        0.0302887 ,  0.01779232, -0.00500663, -0.01324107, -0.00304678])

In [12]:
y = a * (x**2) + b + noisy
y

array([5.49624471, 6.85229113, 6.67646087, 5.9189777 , 5.59075081,
       5.46898213, 6.66706541, 5.30004526, 5.32081308, 5.1205787 ,
       6.15801103, 5.12585385, 6.45347569, 5.23669553, 5.22012467,
       6.22626478, 5.51383899, 5.53876232, 6.1560913 , 5.10331607,
       5.83649308, 5.94042105, 5.38800681, 5.13344202, 5.16922092,
       5.19850636, 8.06163766, 5.55972678, 5.74226104, 6.97296709,
       7.37193286, 6.3902271 , 7.3010453 , 6.0147485 , 5.11180674,
       5.53536473, 5.53904357, 6.0675416 , 5.11651132, 6.61138819,
       5.16925587, 6.11869745, 7.80803996, 8.09035998, 6.86953185,
       5.77273375, 7.86662206, 8.36274476, 8.00577668, 5.11531181])

In [13]:
p_data = pd.DataFrame(data={'x': x, 'y': y})

In [14]:
s.upload_frame(data=p_data, casout=dict(name='train', replace=1))

NOTE: Cloud Analytic Services made the uploaded file available as table TRAIN in caslib CASUSER(weshiz).
NOTE: The table TRAIN has been created in caslib CASUSER(weshiz) from binary data uploaded to Cloud Analytic Services.


CASTable('TRAIN', caslib='CASUSER(weshiz)')

In [15]:
[a, b]

[3.5, 5.1]

### Python Client to check forward and backward

In [18]:
dummy = 0
for i in x:
    print(forward_prop(dummy, dummy, dummy, [i], [a, b], [dummy]))

5.488316699687349
6.850324866270935
6.681167866391715
5.917309837538583
5.59139319853788
5.486986891576672
6.67566727920412
5.305903092007399
5.332565784860756
5.120139319018674
6.162293324414092
5.119758353967989
6.457062258238812
5.225548342167505
5.214175100634697
6.226484580861884
5.51185011230437
5.540270327670545
6.156461842355472
5.10034006863685
5.829952846228029
5.926993795170359
5.372038908444681
5.13513371715279
5.1667730585675
5.1989703765612285
8.080424948857413
5.545279363752737
5.730639616479105
6.991178344688045
7.365617669458094
6.382499505184475
7.307015156238544
6.0043070066505795
5.120306413465875
5.513154803545204
5.544951728310419
6.047268409828857
5.142948148212862
6.615208608186972
5.154733872280525
6.1142827914166675
7.819007684181701
8.094997693254268
6.872613028453529
5.742445055569717
7.848829747503501
8.36775139194212
8.019017754842462
5.1183585893222885


### Train

In [30]:
optimizer=dict(miniBatchSize=10, logLevel=3, 
               maxEpochs=40,# regL2=0.01,
               algorithm=dict(method='VANILLA', 
                              clipGradMax=100, clipGradMin=-100,
                              learningRate=1, gamma=0.1, 
                              lrpolicy='step', stepsize=50
                             )
              )

s.dltrain(table='train', inputs='x',
          target='y',
          modelweights=dict(name='weights', replace=1),
          modeltable='fcmp_model', nthreads=1, recordseed=13309,
          optimizer=optimizer)

NOTE:  Synchronous mode is enabled.
NOTE:  The total number of parameters is 2.
NOTE:  The approximate memory cost is 1.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost       0.01 (s).
NOTE:  The total number of threads on each worker is 1.
NOTE:  The total mini-batch size per thread on each worker is 10.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 10.
NOTE:  Target variable: y
NOTE:  Number of input variables:     1
NOTE:  Number of numeric input variables:      1
NOTE:  Number of FCMP layers in model: 1
NOTE:  FCMP layer 'fcmp_layer' has input tensor size: width=1, height=1, depth=1
NOTE:  FCMP layer 'fcmp_layer' has output tensor size: width=1, height=1, depth=1
NOTE:  FCMP layer 'fcmp_layer' has 2 weights.
NOTE:  Batch nUsed Learning Rate        Loss  Fit Error   Time(s) (Training)
NOTE:      0    10        1            12.95       25.9     0.00
NOTE:      1    10        1           0.2768     0.5535     0.

,Descr,Value
0,Model Name,fcmp_model
1,Model Type,Convolutional Neural Network
2,Number of Layers,3
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,0
6,Number of Pooling Layers,0
7,Number of Fully Connected Layers,0
8,Number of FCMP Layers,1
9,Number of Weight Parameters,2


### Check the weights

In [31]:
s.fetch('weights')

,_LayerID_,_WeightID_,_Weight_
0,1.0,0.0,3.499745
1,1.0,1.0,5.098710


In [48]:
s.endsession()

+ Elapsed: 9.9e-05s, sys: 8.8e-05s, mem: 0.203mb